In [1]:
!pip install langchain langchain-core langchain-community langchain-qdrant langchain-text-splitters fastembed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.5 MB/s eta 0:00:00


In [2]:
import os

pdfNames = [
    "Programming language - Wikipedia.pdf",
    "Computer science - Wikipedia.pdf",
    "Operating system - Wikipedia.pdf",
    "Data structure - Wikipedia.pdf",
    "Algorithm - Wikipedia.pdf",
    "Database - Wikipedia.pdf"
]

paths = [
    os.path.join(
        os.getcwd(),
        filename
    ) for filename in pdfNames
]

print("PDF Paths:\n", paths)

PDF Paths:
 ['/content/Programming language - Wikipedia.pdf', '/content/Computer science - Wikipedia.pdf', '/content/Operating system - Wikipedia.pdf', '/content/Data structure - Wikipedia.pdf', '/content/Algorithm - Wikipedia.pdf', '/content/Database - Wikipedia.pdf']


In [3]:
from langchain_community.document_loaders import PDFPlumberLoader

documents = list()
for path in paths:
    loader = PDFPlumberLoader(path)
    documents.append(loader.load())

documents

[[Document(metadata={'source': '/content/Programming language - Wikipedia.pdf', 'file_path': '/content/Programming language - Wikipedia.pdf', 'page': 0, 'total_pages': 21, 'Title': 'Programming language - Wikipedia', 'Producer': 'macOS Version 15.0 (Build 24A335) Quartz PDFContext', 'Author': 'apple', 'Creator': 'Safari', 'CreationDate': "D:20251117131615Z00'00'", 'ModDate': "D:20251117131615Z00'00'"}, page_content='Programming language\nA programming language is an artificial\nlanguage for expressing computer programs.\n[1]\nProgramming languages typically allow\nsoftware to be written in a human readable\nmanner.\nExecution of a program requires an\nimplementation. There are two main\napproaches for implementing a\nprogramming language – compilation,\nwhere programs are compiled ahead-of-time\nto machine code, and interpretation, where\nprograms are directly executed. In addition\nThe source code for a computer program in C. The\nto these two extremes, some gray lines are comments th

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

textSplitter = RecursiveCharacterTextSplitter(chunk_size=1600, chunk_overlap=250)
allDocuments = list()

for doc in documents:
    chunkedDocs = textSplitter.split_documents(documents = doc)
    allDocuments.extend(chunkedDocs)

print(len(allDocuments))

267


In [ ]:
from langchain_qdrant import FastEmbedSparse, RetrievalMode
from langchain.embeddings import HuggingFaceBgeEmbeddings

modelName = "BAAI/bge-large-en-v1.5"
modelKwargs = {'device': 'cpu'}
encodeKwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=modelName,
    model_kwargs=modelKwargs,
    encode_kwargs=encodeKwargs
)

sparseEmbeddings = FastEmbedSparse(model_name="Qdrant/bm25")

/tmp/ipython-input-1164424153.py:8: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(


Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

arabic.txt: 0.00B [00:00, ?B/s]

danish.txt:   0%|          | 0.00/424 [00:00<?, ?B/s]

french.txt:   0%|          | 0.00/813 [00:00<?, ?B/s]

...

russian.txt: 0.00B [00:00, ?B/s]

norwegian.txt:   0%|          | 0.00/851 [00:00<?, ?B/s]

portuguese.txt: 0.00B [00:00, ?B/s]

swedish.txt:   0%|          | 0.00/559 [00:00<?, ?B/s]

turkish.txt:   0%|          | 0.00/260 [00:00<?, ?B/s]

In [7]:
len(embeddings.embed_query("Hello World"))

1024

In [ ]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
import os

client = QdrantClient(
    url=os.environ.get("QDRANT_URL"),
    api_key=os.environ.get("QDRANT_API_KEY"),
)

vectorStore = QdrantVectorStore(
    client=client,
    collection_name="sampleCollection",
    embedding=embeddings,
    vector_name="semantic-search",
    sparse_vector_name="syntactic-search",
    retrieval_mode=RetrievalMode.SPARSE,
    sparse_embedding=sparseEmbeddings
)

In [10]:
import uuid
ids = [uuid.uuid4() for _ in allDocuments]
vectorStore.add_documents(
    documents=allDocuments,
    ids=ids
)

[UUID('8e97195a-513e-46e4-89e1-30037f5be446'),
 UUID('e0314a62-ba5e-4848-be84-3872ff71fea6'),
 UUID('ccb8c05c-a9c9-4e26-a6e3-576fca000252'),
 UUID('14fed37a-ba7b-4df6-9fc6-c85539151c08'),
 UUID('f8494ca2-507e-4060-9e60-ada88d44bc31'),
 UUID('d01adb4a-167c-48e8-a0d5-b937dc0bacbc'),
 UUID('9355c73d-c61d-4744-ac14-0a11ac0dc68f'),
 UUID('18c3bc64-cb79-4917-ac7a-a68f45d14bc7'),
 UUID('65f0f6af-d173-4792-b661-0a938057aa21'),
 UUID('5a6576ab-4cb7-4c1b-b2b3-5aa64bd0725d'),
 UUID('c036a477-e085-49f8-bf9b-80fcb625293e'),
 UUID('24acf6db-9020-4a3d-a06e-1e214ee61190'),
 UUID('d55e9924-5054-4cb1-9708-c2715ab96714'),
 UUID('099bd464-eab1-42d6-98db-10fba84807b4'),
 UUID('41588e75-b4f5-4e72-b5e1-f33664404cd1'),
 UUID('f57ed191-9129-46ef-acda-ecac7ee8bdc6'),
 UUID('859f62d5-98e8-4d0b-968d-3c93b3c65722'),
 UUID('3773d5b7-5e0f-4415-a64e-1a702279e7e2'),
 UUID('3b0f5bde-6b4a-4605-91db-6427019c1805'),
 UUID('1d0fac5a-ada3-4710-accb-146d9059a433'),
 UUID('4d244a8f-94bb-472d-9f6a-436e925b2084'),
 UUID('cb83cc